<a href="https://colab.research.google.com/github/ChaconLima/mestrado/blob/main/Projeto_3_Introdu%C3%A7%C3%A3o_a_Meta_Heuristicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
#################################################################################################
# library Simulated Annealing - Copyright 2023 Mateus Chacon

# Este programa é um software livre, você pode redistribuí-lo e/ou modificá-lo
# sob os termos da Licença Pública Geral GNU como publicada pela Fundação do Software Livre (FSF),
# na versão 3 da Licença, ou (a seu critério) qualquer versão posterior.

# Este programa é distribuído na esperança de que possa ser útil, mas SEM NENHUMA GARANTIA,
# e sem uma garantia implícita de ADEQUAÇÃO a qualquer MERCADO ou APLICAÇÃO EM PARTICULAR.

# Veja a Licença Pública Geral GNU para mais detalhes
#################################################################################################

!pip install haversine
!pip install pandas

Buscando dados das Localizações

In [90]:
import pandas as pd
import io
import requests

class GetLocation:
  def __init__(self):

    #depricate
    #--------------------------------------------------------------------------
    #url = 'https://drive.google.com/file/d/1dp01gd332PwO2IzQthP9-oJB6Raa2b7g/view?usp=sharing'
    # Fazer a requisição HTTP e obter o conteúdo
    #response = requests.get(url)
    #s = response.content

    # Criar um objeto StringIO a partir do conteúdo obtido
    #file_name = io.StringIO(s.decode('utf-8'))

    # Ler o arquivo CSV utilizando o pandas
    #df_geodata = pd.read_html(file_name)
    #--------------------------------------------------------------------------

    df_geodata = pd.read_csv('municipios.csv')
    df_geodata.columns

    self.geoData = df_geodata

  def getGeoData(self):
    d = self.geoData
    data = {}
    for ad in range(len(d)):
      point = {
          'index': d.values[ad][0],
          'codigo_ibge':d.values[ad][1],
          'nome':d.values[ad][2],
          'latitude':d.values[ad][3],
          'longitude':d.values[ad][4]
      }
      data[ad] = point
    return data

  def getGeoDataByState(self, codState, limit):
    d = self.geoData[self.geoData['codigo_uf'] == codState].copy().reset_index()
    data = {}
    for ad in range(limit):
      point = {
          'index': d.values[ad][0],
          'codigo_ibge':d.values[ad][1],
          'nome':d.values[ad][2],
          'latitude':d.values[ad][3],
          'longitude':d.values[ad][4]
      }
      data[ad] = point
    return data


Classe de Gerenciamento do mapa

In [51]:
import folium

class Map:
  def __init__(self, data = {}):
    self.data = data
    self.map = 0

  def getMap(self):
    return self.map

  def initMap(self):
    coords_brasil = [-16.089461610967666, -55.910299337040286]
    coords_sp = [-23.5509378979565, -46.6398829009542]
    self.map = folium.Map(location= coords_sp, zoom_start=7, tiles="cartodbpositron")

  def createRoute(self, df=[]):
    n_cities = len(df)

    for i in range(len(df)):
      start_point = [self.data[df[i]]['latitude'],self.data[df[i]]['longitude']]
      if (i < (n_cities-1)):
        end_point  = [self.data[df[i+1]]['latitude'],self.data[df[i+1]]['longitude']]
      else:
        end_point  = [self.data[df[0]]['latitude'],self.data[df[0]]['longitude']]

      route = [start_point, end_point]
      folium.PolyLine(route,
                      weight=8,
                      color='red',
                      opacity=0.6).add_to(self.map)

  def drawCities(self,df=[]):
    #For each city draw CircleMarker point in the Map
    for i in range(len(df)):
        folium.CircleMarker(
            location=[self.data[df[i]]['latitude'], self.data[df[i]]['longitude']],
            radius = 6,
            popup=str(i+1)+': '+str(self.data[df[i]]['nome']),
            fill=True, # Set fill to True
            fill_color='lightblue',
            color='grey',
            fill_opacity=0.7
        ).add_to(self.map)

  def renderMap(self, df=[]):
    self.initMap()
    self.drawCities(df)
    self.createRoute(df)
    return self.map

Classe de Gerenciamento da Matriz distancia

In [52]:
import haversine as hs

class Direction():

  def __init__(self, data={}):
    self.directions = {}
    self.data = data
    self.genereteMD()

  def getKey(self, index_origin, index_destination):
    return str(index_origin)+':'+str(index_destination)

  def getMDist(self):
    return self.directions

  def getDistance(self, index_origin, index_destination):
    return self.directions[self.getKey(index_origin,index_destination)]['distance']

  def calcDistence(self, df):
    distance = 0
    for i in range(len(df)-1):
      origin = df[i]
      destination = df[i+1]
      distance += self.getDistance(origin, destination)
    return distance + self.getDistance(df[len(df)-1], df[0])


  def getRoute(self, origin, destination):
    distance  = hs.haversine(origin, destination)
    start_point = origin
    end_point   = destination
    routes = [origin, destination]
    out = {'route':routes,
           'start_point':start_point,
           'end_point':end_point,
           'distance':distance
          }
    return out

  def genereteMD(self):
    keys = list(self.data.keys())
    for i in range(len(keys)):
      for j in range(len(keys)):
        if(i!=j):
          origin = [self.data[keys[i]]['latitude'],self.data[keys[i]]['longitude']]
          destination = [self.data[keys[j]]['latitude'],self.data[keys[j]]['longitude']]
          self.directions[self.getKey(i,j)] = self.getRoute(origin,destination)


##  <b>Template algoritmo do visinho mais próxim.</b>
> 1. Inicialize uma lista vazia de cidades visitadas
> 2. Escolha uma cidade inicial aleatória e adicione-a à lista de cidades visitadas
> 3. Enquanto houver cidades não visitadas:

> > a. Calcule as distâncias entre a cidade atual e todas as cidades não visitadas

> > b. Escolha a cidade mais próxima não visitada e adicione-a à lista de cidades visitadas

> > c. Atualize a cidade atual para a cidade recém-visitada

> 4. Adicione a cidade inicial à lista de cidades visitadas para completar o ciclo
> 5. Retorne a lista de cidades visitadas

In [53]:
import copy

class Answer:
  def __init__(self, answer=[],eval=0):
    self.tour = copy.copy(answer)
    self.distance = copy.copy(eval)

  def getTour(self):
    return self.tour
  def getDistance(self):
    return self.distance


In [54]:
import math

class SolverTspNearest:
  def __init__(self, data, direction):
    self.data = data
    self.direction = direction

  def execute(self):
    num_cities = len(list(self.data))
    visited = [False] * num_cities
    tour = []
    total_distance = 0

    # Start at the first city
    current_city = 0
    tour.append(current_city)
    visited[current_city] = True


    # Repeat until all cities have been visited
    while len(tour) < num_cities:
        nearest_city = None
        nearest_distance = math.inf

        # Find the nearest unvisited city
        for city in range(num_cities):
            if not visited[city]:
                distance = self.direction.getDistance(current_city,city)
                if distance < nearest_distance:
                    nearest_city = city
                    nearest_distance = distance

        # Move to the nearest city
        current_city = nearest_city
        tour.append(current_city)
        visited[current_city] = True
        total_distance += nearest_distance

    # Complete the tour by returning to the starting city
    total_distance += self.direction.getDistance(current_city,0)
    return Answer(tour,total_distance)

##  <b>Template of Simulated Annealing Algorithm.</b>

> <b>Entrada</b>: Esquema de resfriamento. </br>
> s=s0; <i>//Geração da solução pelo algoritmo do visinho mais próximo</i></br>
> T = Tmáx; <i>//Temperatura inicial</i></br>
> <b>Enquanto</b>;</br>
> > <b>Enquanto</b>; <i>//A uma temperatura fixa</i></br>
> > > Gera um vizinho utilizando fleep s';</br>
> > > ΔE = f(s') - f(s);</br>
> > > <b>Se</b> ΔE <= 0 <b>Então</b> s=s' <i>//Aceite a solução vizinha<i></br>
> > > <b>Se não</b> Aceita s' com uma probabilidade <i><b>e^(-ΔE/T)</i></b>;</br>

> > <b>Fim enquanto</b> a condição de Equilíbrio; <i>//por exemplo, um determinado número de iterações executadas em cada temperatura T</i></br>
> > T = g(T); <i>//Atualização de temperatura</i></br>

> <b>Fim enquanto</b> critérios de parada satisfeitos T < Tmin;</br>
> <b>Saída</b>: Melhor solução encontrada.</br>

In [55]:
from math import exp
import random
import copy

class SimulatedAnnealing:

  def __init__(self,data, direction, numberIterations=100, temp=1, seed = 1):
    self.scores=[]
    random.seed(seed)
    self.anwser=Answer
    self.numberIterations = numberIterations
    self.temp = temp
    self.data = data
    self.direction = direction

  def flip(self, route, cutPointer):
    soluction = route[:cutPointer]
    return soluction+route[cutPointer:][::-1]

  def anwserInitial(self):
    return SolverTspNearest(self.data,self.direction).execute()

  def generateAnwserCandidate(self, curr):
    sol = curr.getTour()
    n = len(sol)
    p = random.randint(0, n)
    sol2 = self.flip(sol,p)
    p = random.randint(0, n)
    sol2 = self.flip(sol2,p)
    return Answer(sol2, self.direction.calcDistence(sol2))

  def execute(self):
    best = self.anwserInitial()
    curr = copy.copy(best)
    self.scores.append(best.getDistance())

    for i in range(self.numberIterations):
      candidate = self.generateAnwserCandidate(curr)
      diff = candidate.getDistance() - curr.getDistance()
      if diff < 0:
        diff_global = candidate.getDistance() - best.getDistance()
        if diff_global < 0:
          best = copy.copy(candidate)
        curr = copy.copy(candidate)
        self.scores.append(curr.getDistance())
      else:
        t = self.temp/(i+1)
        prob = exp(-diff/t)
        if random.random() < prob:
          curr  = copy.copy(candidate)

    self.anwser = best
    return best


In [113]:
from datetime import datetime

data = GetLocation()
dataSP = data.getGeoDataByState(35, 600)
direction = Direction(dataSP)

seed = datetime.now().timestamp()
# seed = 1
answer = SimulatedAnnealing(
    data=dataSP,
    direction=direction,
    numberIterations=100,
    temp=1,
    seed=seed).execute()

print("Tour:", answer.getTour())
print("Total distance:",answer.getDistance())

map = Map(dataSP)
map.renderMap(answer.getTour())



Tour: [0, 311, 240, 391, 406, 492, 221, 65, 503, 459, 327, 185, 395, 249, 296, 160, 393, 528, 411, 400, 553, 375, 355, 376, 361, 204, 340, 301, 69, 500, 208, 31, 73, 140, 197, 422, 50, 468, 198, 307, 99, 207, 437, 463, 49, 191, 181, 312, 161, 94, 413, 173, 5, 222, 129, 323, 383, 70, 241, 133, 116, 392, 511, 225, 485, 566, 521, 436, 573, 178, 592, 141, 263, 56, 497, 250, 258, 370, 483, 202, 487, 117, 91, 107, 24, 211, 469, 131, 424, 138, 300, 294, 130, 78, 242, 118, 32, 510, 427, 227, 321, 567, 13, 583, 274, 509, 237, 165, 356, 219, 585, 378, 18, 515, 305, 139, 524, 493, 243, 132, 6, 565, 527, 25, 85, 562, 41, 304, 317, 230, 59, 337, 157, 288, 76, 75, 159, 482, 223, 548, 22, 142, 518, 303, 37, 137, 171, 43, 143, 412, 218, 282, 419, 21, 351, 576, 306, 4, 582, 431, 416, 86, 47, 79, 365, 434, 290, 232, 525, 44, 266, 446, 182, 587, 345, 74, 209, 279, 516, 506, 402, 284, 476, 364, 561, 299, 499, 457, 27, 210, 308, 113, 97, 149, 302, 581, 472, 40, 556, 35, 55, 151, 429, 541, 110, 545, 357, 96